In [23]:
a = {
    "foo": "bar",
    "foo2": "bar2"
}

for key in a.get("foo3", {}).keys():
    print("a")

In [24]:
import http.client
import socket
from pydantic import BaseModel
from typing import Optional, Union, List, Dict, Any
import logging
import json
from urllib.parse import urlparse


class SimpleHttpResponse(BaseModel):
    status: Optional[int]
    decoded_data: Optional[str]
    as_json: Optional[Union[List[Any], Dict[Any, Any]]]
    error: Optional[str]
    timeout: bool

In [55]:
def simple_http_request(
    host: str,
    port: int = http.client.HTTP_PORT,
    path: str = "/",
    timeout: Optional[float] = None,
    method: str = "GET",
    try_json: bool = False,
    follow_redirects: int = 0,
) -> SimpleHttpResponse:
    """This function is a simple wrappper around http.client to make convenient requests and get the answer
    knowing that it will never raise"""

    conn = None
    request_response = SimpleHttpResponse(status=None, decoded_data=None, as_json=None, timeout=False, error=None)

    # Prepare the header for json request
    headers = {}
    if try_json:
        headers["Accept"] = "application/json"

    try:
        # Make the connection and the request
        conn = http.client.HTTPConnection(host, port, timeout=timeout)
        conn.request(method, path, headers=headers)
        response = conn.getresponse()

        # Follow redirects if required
        location_header = response.getheader("location")
        if follow_redirects > 0 and location_header is not None:
            url = urlparse(location_header)
            return simple_http_request(
                host=url.hostname or host,
                port=url.port or port,
                path=url.path or path,  # note: ignoring params, query and fragment
                timeout=timeout,
                method=method,
                try_json=try_json,
                follow_redirects=follow_redirects - 1,
            )

        # Decode the data
        request_response.status = response.status
        if response.status == http.client.OK:
            encoding = response.headers.get_content_charset() or "utf-8"
            request_response.decoded_data = response.read().decode(encoding)

            # Interpret it as json
            if try_json:
                request_response.as_json = json.loads(request_response.decoded_data)

    except socket.timeout as e:
        logging.warning(e)
        request_response.timeout = True
        request_response.error = str(e)

    except (http.client.HTTPException, socket.error, json.JSONDecodeError) as e:
        logging.warning(e)
        request_response.error = str(e)

    except Exception as e:
        logging.error(e, exc_info=True)
        request_response.error = str(e)

    finally:
        if conn:
            conn.close()

    return request_response


simple_http_request("blueos.local", 6020, "/docs", follow_redirects=10, try_json=True)

/docs


SimpleHttpResponse(status=200, decoded_data='<!-- HTML for static distribution bundle build -->\n<!DOCTYPE html>\n<html lang="en">\n  <head>\n    <meta charset="UTF-8">\n    <title>Swagger UI</title>\n    <link rel="stylesheet" type="text/css" href="./swagger-ui.css" />\n    <link rel="icon" type="image/png" href="./favicon-32x32.png" sizes="32x32" />\n    <link rel="icon" type="image/png" href="./favicon-16x16.png" sizes="16x16" />\n    <style>\n      html\n      {\n        box-sizing: border-box;\n        overflow: -moz-scrollbars-vertical;\n        overflow-y: scroll;\n      }\n\n      *,\n      *:before,\n      *:after\n      {\n        box-sizing: inherit;\n      }\n\n      body\n      {\n        margin:0;\n        background: #fafafa;\n      }\n    </style>\n  </head>\n\n  <body>\n    <div id="swagger-ui"></div>\n\n    <script src="./swagger-ui-bundle.js" charset="UTF-8"> </script>\n    <script src="./swagger-ui-standalone-preset.js" charset="UTF-8"> </script>\n    <script>\n    

In [ ]:


# Example:
def main():
    sites = [
        "blueos.local",
        "google.com",
        "example.com",
        "1.1.1.1",
        "8.8.8.8",
        "wikipedia.org",
        "blueos.cloud",
        "yahoo.com",
        "microsoft.com",
    ]
    responses = [simple_http_request(site, path="/", port=80, timeout=0.2) for site in sites]
    [print(response) for response in responses]


# main()

ERROR:root:name 'urljoin' is not defined
Traceback (most recent call last):
  File "/tmp/ipykernel_99351/4032161086.py", line 29, in simple_http_request
    location = urljoin(url, location_header)
NameError: name 'urljoin' is not defined


SimpleHttpResponse(status=None, decoded_data=None, as_json=None, error="name 'urljoin' is not defined", timeout=False)

In [13]:
import asyncio

async def simple_http_request_async(
    host: str,
    port: int = http.client.HTTP_PORT,
    path: str = "/",
    timeout: Optional[float] = None,
    method: str = "GET",
    try_json: bool = False,
) -> SimpleHttpResponse:
    """This function is a simple async wrappper around the simple_http_request"""
    return await asyncio.to_thread(simple_http_request, host=host, port=port, path=path, timeout=timeout, method=method, try_json=try_json)


# Example:
async def main():
    sites = ["blueos.local", "google.com", "example.com", "1.1.1.1", "8.8.8.8", "wikipedia.org", "blueos.cloud", "yahoo.com", "microsoft.com"]
    tasks = [simple_http_request_async(site, path="/", port=80, timeout=1) for site in sites]
    responses = await asyncio.gather(*tasks)
    [print(response) for response in responses]

# Run the asyncio event loop
await main()

status=None decoded_data=None as_json=None error='[Errno -2] Name or service not known' timeout=False
status=301 decoded_data=None as_json=None error=None timeout=False
status=200 decoded_data='<!doctype html>\n<html>\n<head>\n    <title>Example Domain</title>\n\n    <meta charset="utf-8" />\n    <meta http-equiv="Content-type" content="text/html; charset=utf-8" />\n    <meta name="viewport" content="width=device-width, initial-scale=1" />\n    <style type="text/css">\n    body {\n        background-color: #f0f0f2;\n        margin: 0;\n        padding: 0;\n        font-family: -apple-system, system-ui, BlinkMacSystemFont, "Segoe UI", "Open Sans", "Helvetica Neue", Helvetica, Arial, sans-serif;\n        \n    }\n    div {\n        width: 600px;\n        margin: 5em auto;\n        padding: 2em;\n        background-color: #fdfdff;\n        border-radius: 0.5em;\n        box-shadow: 2px 3px 7px 2px rgba(0,0,0,0.02);\n    }\n    a:link, a:visited {\n        color: #38488f;\n        text-dec

In [14]:
from concurrent import futures

# Example:
def main():
    sites = ["blueos.local", "google.com", "example.com", "1.1.1.1", "8.8.8.8", "wikipedia.org", "blueos.cloud", "yahoo.com", "microsoft.com"]
    with futures.ThreadPoolExecutor() as executor:
        tasks = [executor.submit(simple_http_request, site, path="/", port=80, timeout=1) for site in sites]
        responses = [task.result() for task in futures.as_completed(tasks)]
    [print(response) for response in responses]

main()

status=301 decoded_data=None as_json=None error=None timeout=False
status=301 decoded_data=None as_json=None error=None timeout=False
status=200 decoded_data='<!doctype html>\n<html>\n<head>\n    <title>Example Domain</title>\n\n    <meta charset="utf-8" />\n    <meta http-equiv="Content-type" content="text/html; charset=utf-8" />\n    <meta name="viewport" content="width=device-width, initial-scale=1" />\n    <style type="text/css">\n    body {\n        background-color: #f0f0f2;\n        margin: 0;\n        padding: 0;\n        font-family: -apple-system, system-ui, BlinkMacSystemFont, "Segoe UI", "Open Sans", "Helvetica Neue", Helvetica, Arial, sans-serif;\n        \n    }\n    div {\n        width: 600px;\n        margin: 5em auto;\n        padding: 2em;\n        background-color: #fdfdff;\n        border-radius: 0.5em;\n        box-shadow: 2px 3px 7px 2px rgba(0,0,0,0.02);\n    }\n    a:link, a:visited {\n        color: #38488f;\n        text-decoration: none;\n    }\n    @media (